<a href="https://colab.research.google.com/github/ByeonJaeseong/BOJprob/blob/main/Train_modifi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keras-tuner
##!pip install bayesian-optimization

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from kerastuner.tuners import RandomSearch
from google.colab import files
import shutil
from tensorflow.keras.layers import Dropout
#from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LayerNormalization
from google.colab import drive
drive.mount('/content/drive')
#uploaded = files.upload()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.5 MB/s eta 0:00:00


<ipython-input-1-95d6acde1c6f>:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 사용자 정의 Weighted MAPE 손실 함수
def weighted_mape_loss(y_true, y_pred):
    # Calculate the absolute errors and absolute true values
    absolute_errors = tf.abs(y_true - y_pred)
    absolute_true = tf.abs(y_true)

    # Calculate the weights based on the time steps
    weights = tf.linspace(1.0001, 1.1999, tf.shape(y_true)[0])
    weights = tf.expand_dims(weights, axis=-1)  # Expand dimensions to match y_true

     #Calculate the weighted MAPE components

    epsilon = 1e-5  # Small constant to avoid division by zero
    weighted_mape_components = (weights * absolute_errors) / (absolute_true + epsilon)

    weighted_mape = tf.reduce_sum(weighted_mape_components)*100 / tf.cast(tf.shape(y_true)[0], tf.float32)
    return weighted_mape



In [ ]:
# # 사용자 정의 Weighted MAPE 손실 함수
# def weighted_mape_loss(y_true, y_pred):
#     # Calculate the absolute errors and absolute true values
#     absolute_errors = tf.abs(y_true - y_pred)
#     absolute_true = tf.abs(y_true)

#     # Calculate the weights based on the time steps
#     weights = tf.linspace(1.0001, 1.1999, tf.shape(y_true)[0])
#     weights = tf.expand_dims(weights, axis=-1)  # Expand dimensions to match y_true

#     # Calculate the weighted MAPE components
#     weighted_mape_components = weights * absolute_errors / absolute_true

#     # Sum up the weighted components and multiply by 100
#     weighted_mape = tf.reduce_sum(weighted_mape_components)/ tf.cast(tf.shape(y_true)[0], tf.float32) * 100
#     return weighted_mape


In [ ]:
#튜너 삭제
shutil.rmtree('tuner_results')

In [ ]:
#list = ['s30', 's40', 's50', 's70', 's100', 'c30', 'c40', 'c50', 'c70', 'c100'] #모든 파일에 대해서 for 문 돌려서 실행
#count = 1
list = ['c100'] #모든 파일에 대해서 for 문 돌려서 실행
count = 37
for i in list :

    # 데이터 불러오기
    lane_data_c = pd.read_csv('/content/drive/MyDrive/Train/'+'lane_data_c.csv', encoding='utf-8')
    lane_data_s = pd.read_csv('/content/drive/MyDrive/Train/'+'lane_data_s.csv', encoding='utf-8')
    data = pd.read_csv('/content/drive/MyDrive/Train/'+'data_'+i+'.csv', encoding='utf-8') #for문을 이용하여 데이터 바꾸기
    # 데이터 결합
    data_combined = pd.concat([lane_data_c, lane_data_s, data], axis=1) #데이터합치기
    data_combined = data_combined.loc[:, ~data_combined.columns.duplicated()]#거리데이터 중복되어있으니까 빼기
    data_combined = data_combined.drop_duplicates(subset='Distance', keep='first')  # 첫 번째 중복 행만 남기기



    # 입력 변수와 탈선계수 분리
    X_time_series = data_combined[['Distance']]  #Distance를 시계열 데이터로 쓰기 위해서 떼어내기
    X_features = data_combined.drop(['YL_M1_B1_W1', 'YR_M1_B1_W1', 'YL_M1_B1_W2', 'YR_M1_B1_W2', 'Distance'], axis=1) #라벨링데이터 떼어내기
    y = data_combined[['YL_M1_B1_W1', 'YR_M1_B1_W1',
                       'YL_M1_B1_W2', 'YR_M1_B1_W2']] #라벨링하기


    # 데이터 정규화
    scaler = MinMaxScaler() # 스케일링하기
    X_features_scaled = scaler.fit_transform(X_features) #스케일링


    # 학습 데이터와 테스트 데이터 분할
    X_features_train, X_features_test, X_time_series_train, X_time_series_test, y_train, y_test = train_test_split(X_features_scaled, X_time_series, y, test_size=0.2, random_state=42, shuffle=False)


    # 하이퍼파라미터 튜닝을 위한 함수 정의
    def build_model(hp):
        lstm_units = hp.Int('lstm_units', min_value=16, max_value=128, step=8)
        dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.005)
        num_layers = hp.Int('num_layers', min_value=1, max_value=4, step=1)
        optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
        learning_rate = hp.Choice('learning_rate', values=[1e-1,1e-2,1e-3,1e-4])

        input_time_series = Input(shape=(X_time_series_train.shape[1], 1), name='input_time_series')
        input_features = Input(shape=(X_features_train.shape[1],), name='input_features')

        lstm_output = LSTM(units=lstm_units, activation='tanh', return_sequences=True)(input_time_series)
        for _ in range(num_layers):
            lstm_output = LSTM(units=lstm_units, activation='tanh', return_sequences=True)(lstm_output)
            lstm_output = LayerNormalization()(lstm_output)  # Add Layer Normalization here
            lstm_output = Dropout(dropout_rate)(lstm_output)

        lstm_output = LSTM(units=lstm_units, activation='tanh')(lstm_output)

        features_output = Dense(units=32, activation='tanh')(input_features)

        concatenated = concatenate([lstm_output, features_output])

        output_layer = Dense(4)(concatenated)

        tuned_model = Model(inputs=[input_time_series, input_features], outputs=output_layer)
        tuned_model.compile(optimizer=optimizer, loss=weighted_mape_loss)
        return tuned_model

    epochs = 100
    batch_size = 32

    tuner = RandomSearch(build_model, objective='val_loss', max_trials=10, executions_per_trial=3, directory='tuner_results', project_name='model_tuning')

    # 튜닝할 파라미터 정의
    tuner.search_space_summary()

    # 튜닝 실행
    tuner.search([X_time_series_train, X_features_train], y_train, epochs=epochs, batch_size=batch_size, validation_data=([X_time_series_test, X_features_test], y_test))


    # 최적의 모델 선택
    best_model = tuner.get_best_models(num_models=1)[0]

    best_model.fit(
        [X_time_series_train, X_features_train], y_train,
        epochs=100,
        batch_size=32,  # Use the best batch size
        validation_data=([X_time_series_test, X_features_test], y_test),
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
    )

    # 다음 1999개의 샘플 예측
    next_samples = 1999
    X_time_series_predict = X_time_series[-next_samples:]
    X_features_predict = X_features[-next_samples:]

    # 모델 예측
    predictions = best_model.predict([X_time_series_predict, X_features_predict])

    answer_sample = pd.read_csv('/content/drive/MyDrive/Train/'+'answer_sample.csv', header=None)
    answer_sample.iloc[1:, count:count+4] = predictions  # 예측 결과 저장
    answer_sample.to_csv('/content/drive/MyDrive/Train/'+'answer_sample.csv', index=False, header=False)  # 결과를 파일에 저장
    count = count + 4
    #튜너 삭제
    shutil.rmtree('tuner_results')

Trial 10 Complete [00h 31m 11s]
val_loss: 140283.31510416666

Best val_loss So Far: 77219.86588541667
Total elapsed time: 03h 28m 34s
Epoch 1/100
300/300 [==============================] - 10s 15ms/step - loss: 1363.0514 - val_loss: 294734.0312
Epoch 2/100
300/300 [==============================] - 4s 13ms/step - loss: 1408.5605 - val_loss: 515463.0312
Epoch 3/100
300/300 [==============================] - 3s 11ms/step - loss: 1415.1862 - val_loss: 118918.5703
Epoch 4/100
300/300 [==============================] - 3s 10ms/step - loss: 1523.2458 - val_loss: 475146.7188
Epoch 5/100
300/300 [==============================] - 3s 9ms/step - loss: 1496.0929 - val_loss: 242553.9219
Epoch 6/100
300/300 [==============================] - 3s 10ms/step - loss: 1329.6841 - val_loss: 414044.0938
Epoch 7/100
300/300 [==============================] - 4s 14ms/step - loss: 1267.6819 - val_loss: 136873.9531
Epoch 8/100
63/63 [==============================] - 1s 3ms/step


In [ ]:

answer_sample = pd.read_csv('/content/drive/MyDrive/Train/'+'answer_sample.csv', header=None)
answer_sample.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,Distance,YL_M1_B1_W1_s30,YR_M1_B1_W1_s30,YL_M1_B1_W2_s30,YR_M1_B1_W2_s30,YL_M1_B1_W1_s40,YR_M1_B1_W1_s40,YL_M1_B1_W2_s40,YR_M1_B1_W2_s40,YL_M1_B1_W1_s50,...,YL_M1_B1_W2_c50,YR_M1_B1_W2_c50,YL_M1_B1_W1_c70,YR_M1_B1_W1_c70,YL_M1_B1_W2_c70,YR_M1_B1_W2_c70,YL_M1_B1_W1_c100,YR_M1_B1_W1_c100,YL_M1_B1_W2_c100,YR_M1_B1_W2_c100
1,2500.25,-0.005451131612062454,-0.01783442124724388,0.03200285881757736,0.05779584124684334,-0.04307665303349495,-0.09566867351531982,0.21014060080051422,0.391579806804657,0.3838177025318146,...,-0.005846066400408745,0.007688150741159916,-0.12530751526355743,-0.07672007381916046,0.11807942390441895,0.1566709280014038,0.21684807538986206,-0.1546158641576767,-0.051741790026426315,-0.13296422362327576
2,2500.5,-0.006047192960977554,-0.030401546508073807,0.03775135427713394,0.07066535204648972,-0.01450737938284874,0.0404275506734848,0.011880887672305107,0.021699300035834312,-0.006313069723546505,...,0.00240144319832325,0.005255519412457943,-0.013455910608172417,0.0048828450962901115,0.03649358078837395,0.015872806310653687,-0.01047162339091301,-0.017726225778460503,0.010488692671060562,-0.05979997664690018
3,2500.75,-0.004779655486345291,-0.03646775335073471,0.04574955254793167,0.06768506020307541,-0.021262388676404953,0.053688716143369675,0.013380570337176323,0.03310257941484451,0.0018084505572915077,...,0.01112205721437931,-0.030324582010507584,-0.02297811023890972,-0.0027162106707692146,0.04607218876481056,0.028446724638342857,-0.027313221246004105,-0.014971328899264336,-0.009672697633504868,-0.09834503382444382
4,2501.0,0.00019299611449241638,-0.033213354647159576,0.05911453813314438,0.055825281888246536,-0.01840103790163994,0.056864019483327866,0.007383702322840691,0.037452273070812225,0.0021323421970009804,...,0.01629762537777424,-0.04650920256972313,-0.0269708763808012,0.0053975461050868034,0.020965492352843285,0.02843737043440342,-0.022244427353143692,-0.014987332746386528,-0.02368360385298729,-0.09130804985761642
